*아래 링크를 통해 이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.*

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nlp-with-pytorch/blob/master/chapter_5/5_1_Pretrained_Embeddings.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nlp-with-pytorch/blob/master/chapter_5/5_1_Pretrained_Embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from annoy import AnnoyIndex
import numpy as np

In [2]:
class PreTrainedEmbeddings(object):
    """ 사전 훈련된 단어 벡터 사용을 위한 래퍼 클래스 """
    def __init__(self, word_to_index, word_vectors):
        """
        매개변수:
            word_to_index (dict): 단어에서 정수로 매핑
            word_vectors (numpy 배열의 리스트)
        """
        self.word_to_index = word_to_index
        self.word_vectors = word_vectors
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}

        self.index = AnnoyIndex(len(word_vectors[0]), metric='euclidean')
        print("인덱스 만드는 중!")
        for _, i in self.word_to_index.items():
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print("완료!")
        
    @classmethod
    def from_embeddings_file(cls, embedding_file):
        """사전 훈련된 벡터 파일에서 객체를 만듭니다.
        
        벡터 파일은 다음과 같은 포맷입니다:
            word0 x0_0 x0_1 x0_2 x0_3 ... x0_N
            word1 x1_0 x1_1 x1_2 x1_3 ... x1_N
        
        매개변수:
            embedding_file (str): 파일 위치
        반환값:
            PretrainedEmbeddings의 인스턴스
        """
        word_to_index = {}
        word_vectors = []

        with open(embedding_file) as fp:
            for line in fp.readlines():
                line = line.split(" ")
                word = line[0]
                vec = np.array([float(x) for x in line[1:]])
                
                word_to_index[word] = len(word_to_index)
                word_vectors.append(vec)
                
        return cls(word_to_index, word_vectors)
    
    def get_embedding(self, word):
        """
        매개변수:
            word (str)
        반환값
            임베딩 (numpy.ndarray)
        """
        return self.word_vectors[self.word_to_index[word]]

    def get_closest_to_vector(self, vector, n=1):
        """벡터가 주어지면 n 개의 최근접 이웃을 반환합니다
        매개변수:
            vector (np.ndarray): Annoy 인덱스에 있는 벡터의 크기와 같아야 합니다
            n (int): 반환될 이웃의 개수
        반환값:
            [str, str, ...]: 주어진 벡터와 가장 가까운 단어
                단어는 거리순으로 정렬되어 있지 않습니다.
        """
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def compute_and_print_analogy(self, word1, word2, word3):
        """단어 임베딩을 사용한 유추 결과를 출력합니다

        word1이 word2일 때 word3은 __입니다.
        이 메서드는 word1 : word2 :: word3 : word4를 출력합니다
        
        매개변수:
            word1 (str)
            word2 (str)
            word3 (str)
        """
        vec1 = self.get_embedding(word1)
        vec2 = self.get_embedding(word2)
        vec3 = self.get_embedding(word3)

        # 네 번째 단어 임베딩을 계산합니다
        spatial_relationship = vec2 - vec1
        vec4 = vec3 + spatial_relationship

        closest_words = self.get_closest_to_vector(vec4, n=4)
        existing_words = set([word1, word2, word3])
        closest_words = [word for word in closest_words 
                             if word not in existing_words] 

        if len(closest_words) == 0:
            print("계산된 벡터와 가장 가까운 이웃을 찾을 수 없습니다!")
            return
        
        for word4 in closest_words:
            print("{} : {} :: {} : {}".format(word1, word2, word3, word4))

In [3]:
embeddings = PreTrainedEmbeddings.from_embeddings_file('data/glove/glove.6B.100d.txt')

Building Index!
Finished!


In [4]:
embeddings.compute_and_print_analogy('man', 'he', 'woman')

man : he :: woman : she
man : he :: woman : never
man : he :: woman : her


In [5]:
embeddings.compute_and_print_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship
fly : plane :: sail : vessel


In [6]:
embeddings.compute_and_print_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : puppy
cat : kitten :: dog : puppies
cat : kitten :: dog : toddler


In [7]:
embeddings.compute_and_print_analogy('blue', 'color', 'dog')

blue : color :: dog : pig
blue : color :: dog : typical
blue : color :: dog : adult
blue : color :: dog : and/or


In [8]:
embeddings.compute_and_print_analogy('leg', 'legs', 'hand')

leg : legs :: hand : hands
leg : legs :: hand : ears
leg : legs :: hand : eyes


In [9]:
embeddings.compute_and_print_analogy('toe', 'foot', 'finger')

toe : foot :: finger : hand
toe : foot :: finger : kept
toe : foot :: finger : ground


In [10]:
embeddings.compute_and_print_analogy('talk', 'communicate', 'read')

talk : communicate :: read : correctly
talk : communicate :: read : instructions


In [11]:
embeddings.compute_and_print_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican
blue : democrat :: red : congressman
blue : democrat :: red : senator


In [12]:
embeddings.compute_and_print_analogy('man', 'king', 'woman')

man : king :: woman : queen
man : king :: woman : throne
man : king :: woman : elizabeth


In [13]:
embeddings.compute_and_print_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse
man : doctor :: woman : physician
man : doctor :: woman : pregnant


In [14]:
embeddings.compute_and_print_analogy('fast', 'fastest', 'small')

fast : fastest :: small : ten
fast : fastest :: small : registered
fast : fastest :: small : eight
